In [ ]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

f_data_loading <- "./data_loading.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_data_loading, '--to=script', '--stdout'), stdout = TRUE)))

In [ ]:
options(repr.plot.width=7, repr.plot.height=5)
for(sel_sample in c("plasma", "BAL")[2]){
    Afum_length <- Afum_IS_meta %>% 
        filter(read == "R1") %>% 
        filter(origin != "controls") %>% 
        filter(type != "ds-wcDNA") %>%    
        mutate(TLEN = abs(TLEN)) %>% 
        filter(sample == sel_sample) %>% 
        group_by(sample, type, sample_type, TLEN) %>% 
        summarise(sumCount = sum(Count), .groups = "keep") %>% 
        group_by(sample, type, sample_type) %>% 
        mutate(norm_sumCount = sumCount/sum(sumCount)*100) %>% 
        mutate(sample_type = factor(sample_type, levels = c("dsLP plasma", "ssLP plasma", 
                                                            "dsLP BAL", "ssLP BAL"))) %>%  
        ggplot(aes(x = TLEN, y = norm_sumCount, col = sample_type)) + 
            geom_segment(aes(x = TLEN, xend = TLEN, y = 0, yend = norm_sumCount), size = 1) + 
            facet_grid(rows = vars(type), drop = TRUE, scales = "free") + 
            scale_color_manual(values = c("dsLP plasma" = "#C6896C", "ssLP plasma" = "#A84750", 
                                             "dsLP BAL" = "#7CA2C2", "ssLP BAL" = "#6066B6")) + 
            xlab("Template length (bp)") + ylab("Percentage of fragments (%)") + labs(color = "") + 
            geom_text(data = . %>%
                      summarise(sumsumCount = sum(sumCount), .groups = "drop") %>%
                      mutate(label = paste("# reads = ", sumsumCount)),  # Create label text
                  aes(x = Inf, y = Inf, col = sample_type, label = label),  # Position text at the top-right corner
                  hjust = 1.5, vjust = 1.5, size = 3.5) +
            theme_bw() + 
            theme(axis.text.x = element_text(angle = 90, hjust = 1), 
                strip.background = element_rect(color = "white", fill = "white"), 
                strip.text.x = element_blank(), 
                panel.spacing = unit(1, "lines"), 
                strip.text.y = element_blank()) 

    Afum_length %>% print()

    ggsave(paste0("../../output_figures/Afumigatus_", sel_sample, "_length.png"), plot = Afum_length, 
                width = 7, height = 5)
    ggsave(paste0("../../output_figures/Afumigatus_", sel_sample, "_length.pdf"), plot = Afum_length, 
                width = 7, height = 5)
}

In [ ]:
freq_tbl_Afum_IS_meta <- Afum_IS_meta %>% 
        #filter(read == "R1") %>% 
        filter(origin != "controls") %>% 
        filter(type != "ds-wcDNA") %>%    
        mutate(TLEN = abs(TLEN)) %>% 
        group_by(sample, type, sample_type, TLEN) %>% 
        summarise(sumCount = sum(Count), .groups = "keep") 

expanded_tbl_Afum_IS_meta <- freq_tbl_Afum_IS_meta %>% 
    uncount(weights = sumCount) 


quartiles_tlen <- expanded_tbl_Afum_IS_meta %>%
    group_by(sample_type) %>% 
    summarise(
        Q1 = quantile(TLEN, 0.25, na.rm = TRUE),
        Q2 = quantile(TLEN, 0.50, na.rm = TRUE),  # Median
        Q3 = quantile(TLEN, 0.75, na.rm = TRUE),
        .groups = "drop"
    )
quartiles_tlen